## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# ToDo encode this differently
attrition_df["OverTime"].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [4]:
# ToDo Encode this differently (ordinally?)
attrition_df["PerformanceRating"].value_counts()

PerformanceRating
3    1244
4     226
Name: count, dtype: int64

attrition_df.columns

In [5]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

In [6]:
# Using these target columns later
target_columns = ['Attrition', 'Department']

In [7]:
# Get the columns with data types other than 'int64'
non_int_columns = attrition_df.select_dtypes(exclude='int64').columns.tolist()
print(non_int_columns)

['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime']


In [8]:
# Important for later
try:
    non_int_columns.remove("Attrition")
except ValueError:
    pass
try:
    non_int_columns.remove("Department")
except ValueError:
    pass

In [9]:
from sklearn.preprocessing import OneHotEncoder
# Initialize OneHotEncoder because categories are not ordinal
encoder = OneHotEncoder()
# Extract the columns to be encoded from your DataFrame
columns_to_encode_df = attrition_df[non_int_columns]

In [10]:
# Fit and transform the categorical column
encoded_features = encoder.fit_transform(columns_to_encode_df)
print(encoded_features)

  (0, 2)	1.0
  (0, 4)	1.0
  (0, 16)	1.0
  (0, 20)	1.0
  (0, 22)	1.0
  (1, 1)	1.0
  (1, 4)	1.0
  (1, 15)	1.0
  (1, 19)	1.0
  (1, 21)	1.0
  (2, 2)	1.0
  (2, 7)	1.0
  (2, 11)	1.0
  (2, 20)	1.0
  (2, 22)	1.0
  (3, 1)	1.0
  (3, 4)	1.0
  (3, 15)	1.0
  (3, 19)	1.0
  (3, 22)	1.0
  (4, 2)	1.0
  (4, 6)	1.0
  (4, 11)	1.0
  (4, 19)	1.0
  (4, 21)	1.0
  :	:
  (1465, 1)	1.0
  (1465, 6)	1.0
  (1465, 11)	1.0
  (1465, 19)	1.0
  (1465, 21)	1.0
  (1466, 2)	1.0
  (1466, 6)	1.0
  (1466, 9)	1.0
  (1466, 19)	1.0
  (1466, 21)	1.0
  (1467, 2)	1.0
  (1467, 4)	1.0
  (1467, 13)	1.0
  (1467, 19)	1.0
  (1467, 22)	1.0
  (1468, 1)	1.0
  (1468, 6)	1.0
  (1468, 16)	1.0
  (1468, 19)	1.0
  (1468, 21)	1.0
  (1469, 2)	1.0
  (1469, 6)	1.0
  (1469, 11)	1.0
  (1469, 19)	1.0
  (1469, 21)	1.0


In [11]:
# Convert the encoded features to a DataFrame
encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoder.get_feature_names_out(non_int_columns))
encoded_df.head()

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [12]:
# Concatenate the encoded DataFrame with the original DataFrame
attrition_df_encoded = pd.concat([attrition_df.drop(non_int_columns, axis=1), encoded_df], axis=1)
attrition_df_encoded.head()

,Age,Attrition,Department,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,41,Yes,Sales,1,2,2,94,3,2,4,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,49,No,Research & Development,8,1,3,61,2,2,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,37,Yes,Research & Development,2,2,4,92,2,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,33,No,Research & Development,3,4,4,56,3,1,3,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,27,No,Research & Development,2,1,1,40,3,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [13]:
attrition_df_encoded.columns

Index(['Age', 'Attrition', 'Department', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'NumCompaniesWorked', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'BusinessTravel_Non-Travel',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'EducationField_Human Resources', 'EducationField_Life Sciences',
       'EducationField_Marketing', 'EducationField_Medical',
       'EducationField_Other', 'EducationField_Technical Degree',
       'JobRole_Healthcare Representative', 'JobRole_Human Resources',
       'JobRole_Laboratory Technician', 'JobRole_Manager',
       'JobRole_Manufacturing Director', 'JobRole_Research Director',
       'JobRole_Resear

In [14]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df_encoded[target_columns]

In [15]:
# # see unique values of attrition column
# attrition_df["Attrition"].value_counts()

In [16]:
# # make sure the attrition column is numerical
# def binaryEncode(value):
#     if value == "Yes":
#         return 1
#     if value == "No":
#         return 0
#     else:
#         raise Exception("An error occurred")

# attrition_df["Attrition"] = attrition_df["Attrition"].apply(binaryEncode)
# attrition_df.head()

In [17]:
# Going to turn the y_df into numerical data

In [18]:
# Create a list of at least 10 column names to use as X data
copy_df = attrition_df_encoded.copy()
copy_df = copy_df.drop(target_columns, axis=1)
X_columns = copy_df.columns

# Create X_df using your selected columns
X_df = attrition_df_encoded[X_columns]

# Show the data types for X_df
print(X_df.dtypes)


Age                                    int64
DistanceFromHome                       int64
Education                              int64
EnvironmentSatisfaction                int64
HourlyRate                             int64
JobInvolvement                         int64
JobLevel                               int64
JobSatisfaction                        int64
NumCompaniesWorked                     int64
PercentSalaryHike                      int64
PerformanceRating                      int64
RelationshipSatisfaction               int64
StockOptionLevel                       int64
TotalWorkingYears                      int64
TrainingTimesLastYear                  int64
WorkLifeBalance                        int64
YearsAtCompany                         int64
YearsInCurrentRole                     int64
YearsSinceLastPromotion                int64
YearsWithCurrManager                   int64
BusinessTravel_Non-Travel            float64
BusinessTravel_Travel_Frequently     float64
BusinessTr

In [19]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [20]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
print(y_train["Department"])

1343    Research & Development
1121                     Sales
1048                     Sales
1393                     Sales
527                      Sales
                 ...          
1130    Research & Development
1294    Research & Development
860     Research & Development
1459    Research & Development
1126                     Sales
Name: Department, Length: 1102, dtype: object


In [22]:
# Create a OneHotEncoder for the Department column
one_hot_encoder_department = OneHotEncoder()

# Fit the encoder to the training data
one_hot_encoder_department.fit(y_train["Department"].values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = one_hot_encoder_department.transform(y_train["Department"].values.reshape(-1, 1)).toarray()
y_test_department_encoded = one_hot_encoder_department.transform(y_test["Department"].values.reshape(-1, 1)).toarray()



In [23]:
y_train_department_df = pd.DataFrame(y_train_department_encoded)
y_train_department_df.head()

,0,1,2
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0


In [24]:
y_test_department_df = pd.DataFrame(y_test_department_encoded)
y_test_department_df.head()

,0,1,2
0,0.0,0.0,1.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0


In [25]:
# Create a OneHotEncoder for the Attrition column
one_hot_encoder_attrition = OneHotEncoder()

# Fit the encoder to the training data
one_hot_encoder_attrition.fit(y_train["Attrition"].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = one_hot_encoder_attrition.transform(y_train["Attrition"].values.reshape(-1, 1)).toarray()
y_test_attrition_encoded = one_hot_encoder_attrition.transform(y_test["Attrition"].values.reshape(-1, 1)).toarray()

In [26]:
y_train_attrition_df = pd.DataFrame(y_train_attrition_encoded)
y_train_attrition_df.head()

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [27]:
y_test_attrition_df = pd.DataFrame(y_test_attrition_encoded)
y_test_attrition_df.head()

,0,1
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0


## Create, Compile, and Train the Model

In [28]:
train_df = pd.DataFrame(X_train)
train_df.head()

,Age,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,PercentSalaryHike,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
1343,29,7,3,4,59,3,1,1,3,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1121,36,1,4,2,73,3,2,3,6,13,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1048,34,3,3,4,81,1,2,1,3,18,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1393,27,9,3,4,44,3,2,4,1,14,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
527,32,10,3,4,55,3,2,4,1,12,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [29]:
from tensorflow.keras.layers import Input, Dense
# Find the number of columns in the X training data
num_input_features = len(train_df.columns)
print(num_input_features)
# Create the input layer
input_layer = Input(shape=(num_input_features,))
# Create at least two shared layers
shared_layer1 = Dense(64, activation='relu')(input_layer)
shared_layer2 = Dense(32, activation='relu')(shared_layer1)

43


In [30]:
num_department_classes = attrition_df["Department"].value_counts()
print(len(num_department_classes))

3


In [31]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = Dense(64, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = Dense(len(num_department_classes), activation='softmax', name='department_output')(department_hidden_layer)


In [32]:
num_attrition_classes = attrition_df["Attrition"].value_counts()
print(len(num_attrition_classes))

2


In [33]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = Dense(64, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = Dense(len(num_attrition_classes), activation='softmax', name='attrition_output')(attrition_hidden_layer)



In [34]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer=Adam(), loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'}, metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 43)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      2,816 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      2,112 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │      2,112 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │        195 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │        130 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,445 (36.89 KB)

 Trainable params: 9,445 (36.89 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Debugging
print("Shape of y_train_department_encoded:", y_train_department_encoded.shape)
print("Shape of y_train_attrition_encoded:", y_train_attrition_encoded.shape)

Shape of y_train_department_encoded: (1102, 3)
Shape of y_train_attrition_encoded: (1102, 2)


In [36]:
# Debuggin Continued
print("Data type of y_train_department_encoded:", y_train_department_encoded.dtype)
print("Data type of y_train_attrition_encoded:", y_train_attrition_encoded.dtype)

Data type of y_train_department_encoded: float64
Data type of y_train_attrition_encoded: float64


In [37]:
print("Sample values of y_train_department_encoded:", y_train_department_encoded[:10])
print("Sample values of y_train_attrition_encoded:", y_train_attrition_encoded[:10])

Sample values of y_train_department_encoded: [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]
Sample values of y_train_attrition_encoded: [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [38]:
# More Debugging
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train_department_encoded.shape)
print(y_train_attrition_encoded.shape)
print(y_test_department_encoded.shape)
print(y_test_attrition_encoded.shape)

(1102, 43)
(368, 43)
(1102, 3)
(1102, 2)
(368, 3)
(368, 2)


In [39]:
epochs = 20
batch_size = 32
# Train the model
history = model.fit(X_train_scaled, {'department_output': y_train_department_encoded, 'attrition_output': y_train_attrition_encoded}, 
                    validation_data=(X_test_scaled, {'department_output': y_test_department_encoded, 'attrition_output': y_test_attrition_encoded}), 
                    epochs=epochs, batch_size=batch_size, verbose=2)

Epoch 1/20


ValueError: For a model with multiple outputs, when providing the `metrics` argument as a list, it should have as many entries as the model has outputs. Received:
metrics=['accuracy']
of length 1 whereas the model has 2 outputs.

In [ ]:
# Evaluate the model with the testing data


In [ ]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 